# Extract vertical cross section for Rio Negro

In [128]:
import xarray as xr 
import numpy as np
from pathlib import Path 
import wrf 
from netCDF4 import Dataset
import calendar 
import pandas as pd 

In [132]:
def create_hourly_time_range(year, month):
    """
    Create a time range with hourly values for a given month and year.

    Parameters:
        year (int): The year (e.g., 2023).
        month (int): The month (e.g., 1 for January).

    Returns:
        pd.DatetimeIndex: A time range with hourly values for the given month.
    """
    # Determine the number of days in the month
    num_days = calendar.monthrange(year, month)[1]

    # Create a time range with hourly frequency
    start_date = f'{year}-{month:02d}-01'
    end_date = f'{year}-{month:02d}-{num_days:02d} 23:59:59'
    time_range = pd.date_range(start=start_date, end=end_date, freq='3h')

    return time_range

### Look at land surface data - how wide are the rivers? 

In [20]:
path = Path('/glade/campaign/univ/uiuc0017/chliu/WRF4KM_2000-2020/wrf2d_wrf3d/') 
wrf_constants = xr.open_dataset(path / 'wrfconstants_SAAG_20yr.nc').squeeze()      

In [149]:
wrf_constants.XLAT.dims

('south_north', 'west_east')

In [51]:
land_mask = wrf_constants.LANDMASK
land_data = wrf_constants.XLAND
lake_mask = wrf_constants.LAKEMASK

# cross section coordinates 
lat_values = wrf_constants.XLAT[south_north_inds, west_east_inds]
lon_values = wrf_constants.XLONG[south_north_inds, west_east_inds]

In [ ]:
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs
import cartopy.feature as cfeature

amazon_lat_min = -8
amazon_lat_max = 3
amazon_lon_min = -70
amazon_lon_max = -55
fs = 24 

fig = plt.figure(figsize=(20, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.set_extent([amazon_lon_min, amazon_lon_max, amazon_lat_min, amazon_lat_max]) 
mask=ax.pcolormesh(land_mask.XLONG, land_mask.XLAT, land_mask, transform=ccrs.PlateCarree(), vmin = 0, vmax = 1 )

ax.plot(lon_values, lat_values, color='teal', linewidth=1.0,  transform=ccrs.PlateCarree(), label = 'Rio Negro')
gl = ax.gridlines(draw_labels=True, linewidth=0.001, color='gray', linestyle='--')
plt.title('Land mask', fontsize = fs )

plt.savefig('land_mask_and_cross_section.png', bbox_inches = 'tight')
plt.show()

In [12]:
# Define the indices for the cross-section
west_east_inds = np.array([
    564, 564, 564, 564, 564, 564, 564, 564, 564, 564, 564, 564, 564,
    564, 564, 564, 564, 564, 564, 564, 564, 564, 564, 564, 564, 564,
    564, 563, 563, 563, 563, 563, 563, 563, 563, 563, 563, 563, 563,
    563, 563, 563, 563, 563, 563, 563, 563, 563, 563, 563
])

south_north_inds = np.array([
    1577, 1578, 1579, 1580, 1581, 1582, 1583, 1583, 1584, 1585, 1586,
    1587, 1588, 1589, 1590, 1590, 1591, 1592, 1593, 1594, 1595, 1596,
    1597, 1597, 1598, 1599, 1600, 1601, 1602, 1603, 1604, 1604, 1605,
    1606, 1607, 1608, 1609, 1610, 1611, 1611, 1612, 1613, 1614, 1615,
    1616, 1617, 1618, 1618, 1619, 1620
])

In [77]:
file_path = path / '2020' /  'wrf3d_d01_2020-12-31_090000'  
ds = xr.open_dataset(file_path)
wrfin = Dataset(file_path)

#vwind=wrf.getvar(wrfin,'va')
#uwind=wrf.getvar(wrfin,'ua')
#cape = wrf.getvar(wrfin, 'cape_3d')

In [118]:
from tqdm import tqdm 

def get_cross_section(year, month, south_north_inds = south_north_inds, west_east_inds  = west_east_inds):
    '''
    Function to extract cross section along a given line of indices

    Returns: 
        Dataset with variables along cross sections for one month. Dims: bottom_top, points, time

    '''
    path = Path('/glade/campaign/univ/uiuc0017/chliu/WRF4KM_2000-2020/wrf2d_wrf3d/') 
    vars = ['QRAIN', 'QGRAUP', 'QSNOW', 'QCLOUD', 'QVAPOR', 'QICE', 'TK','P','U', 'V']
    year = str(year) 
    month = str(month).zfill(2)
    # open 3D data for WRF timestep 
    file_paths = list((path / year ).glob( str('wrf3d_d01_'+year+'-'+month+'*')))
    file_paths.sort
    print(len(file_paths), 'files for ', year, month, flush = True)
    for fname in tqdm(file_paths):
        ds = xr.open_dataset(fname).squeeze()
    
        # extract the cross-section for unstaggered variables
        selected_data = {} 
        for var in vars:
            staggered_data = ds[var]
            if 'west_east_stag' in staggered_data.dims:
                selected_data[var] = ds[var].isel(west_east_stag= xr.DataArray(west_east_inds, dims = 'points'),
                              south_north= xr.DataArray(south_north_inds, dims = 'points'))
            elif 'south_north_stag' in staggered_data.dims:
                selected_data[var] = ds[var].isel(west_east= xr.DataArray(west_east_inds, dims = 'points'),
                              south_north_stag= xr.DataArray(south_north_inds, dims = 'points'))
            else:
                selected_data[var] = ds[var].isel(west_east= xr.DataArray(west_east_inds, dims = 'points'),
                                                  south_north= xr.DataArray(south_north_inds, dims = 'points'))
        if fname == file_paths[0]:
            cross_section = xr.Dataset(selected_data)
        else:
            cross_section = xr.concat([cross_section, xr.Dataset(selected_data)], dim = 'time') 

    return cross_section 

In [119]:
year = '2020'
for mon in np.arange(7,8):
    month = str(mon).zfill(2)
    cross_section = get_cross_section(year, mon)
    
    # add time dimension and coordinates 
    cross_section['time'] = create_hourly_time_range(int(year), int(month))
    cross_section['lat'] = lat_values
    cross_section['lon'] = lon_values

    ### save netcdf file ### 
    output_file = Path('data') / str('rio_negro_' + year + '_' + month+ '.nc')  
    cross_section.to_netcdf(output_file)

240 files for  2020 06


100%|██████████| 240/240 [09:08<00:00,  2.29s/it]


In [161]:
test_time_series = xr.open_dataset(output_file) 
test_time_series

<xarray.Dataset> Size: 29MB
Dimensions:  (time: 240, bottom_top: 60, points: 50, south_north: 50,
              west_east: 50)
Coordinates:
    XTIME    (time) datetime64[ns] 2kB ...
  * time     (time) datetime64[ns] 2kB 2020-06-01 ... 2020-06-30T21:00:00
    XLAT     (south_north, west_east) float32 10kB ...
    XLONG    (south_north, west_east) float32 10kB ...
Dimensions without coordinates: bottom_top, points, south_north, west_east
Data variables:
    QRAIN    (time, bottom_top, points) float32 3MB ...
    QGRAUP   (time, bottom_top, points) float32 3MB ...
    QSNOW    (time, bottom_top, points) float32 3MB ...
    QCLOUD   (time, bottom_top, points) float32 3MB ...
    QVAPOR   (time, bottom_top, points) float32 3MB ...
    QICE     (time, bottom_top, points) float32 3MB ...
    TK       (time, bottom_top, points) float32 3MB ...
    P        (time, bottom_top, points) float32 3MB ...
    U        (time, bottom_top, points) float32 3MB ...
    V        (time, bottom_top, points) float32 3MB ...
    lat      (south_north, west_east) float32 10kB ...
    lon      (south_north, west_east) float32 10kB ...

In [160]:
output_file

PosixPath('data/rio_negro_2020_06.nc')

In [ ]:
#### unstagger grid first 

cross_section_staggered = dict()
for var in staggered_vars:
    staggered_data = ds[var]
    if 'west_east_stag' in staggered_data.dims:
        staggered_data = staggered_data.interp(west_east_stag=ds['west_east'])
    if 'south_north_stag' in staggered_data.dims:
        staggered_data = staggered_data.interp(south_north_stag=ds['south_north'])
    
    cross_section_staggered[var] = staggered_data.isel(
        west_east=xr.DataArray(west_east_inds, dims='points'),
        south_north=xr.DataArray(south_north_inds, dims='points'))